In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [8]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 173ms/step - loss: 3.1377 - accuracy: 0.5378
Epoch 2/30
58/58 [==============================] - 10s 177ms/step - loss: 2.0994 - accuracy: 0.5926
Epoch 3/30
58/58 [==============================] - 10s 171ms/step - loss: 1.6657 - accuracy: 0.6594
Epoch 4/30
58/58 [==============================] - 10s 174ms/step - loss: 1.4455 - accuracy: 0.7130
Epoch 5/30
58/58 [==============================] - 10s 177ms/step - loss: 1.2849 - accuracy: 0.7492
Epoch 6/30
58/58 [==============================] - 10s 177ms/step - loss: 1.1579 - accuracy: 0.7711
Epoch 7/30
58/58 [==============================] - 10s 176ms/step - loss: 1.1089 - accuracy: 0.7985
Epoch 8/30
58/58 [==============================] - 10s 176ms/step - loss: 0.9755 - accuracy: 0.8226
Epoch 9/30
58/58 [==============================] - 10s 176ms/step - loss: 0.9175 - accuracy: 0.8226
Epoch 10/30
58/58 [==============================] - 10s 174ms/step - loss: 0.9661 - accura

58/58 [==============================] - 10s 175ms/step - loss: 0.8275 - accuracy: 0.5465
Epoch 22/30
58/58 [==============================] - 10s 178ms/step - loss: 0.8126 - accuracy: 0.5641
Epoch 23/30
58/58 [==============================] - 10s 179ms/step - loss: 0.7986 - accuracy: 0.5805
Epoch 24/30
58/58 [==============================] - 10s 177ms/step - loss: 0.7855 - accuracy: 0.5827
Epoch 25/30
58/58 [==============================] - 10s 175ms/step - loss: 0.7695 - accuracy: 0.6046
Epoch 26/30
58/58 [==============================] - 10s 175ms/step - loss: 0.7509 - accuracy: 0.6539
Epoch 27/30
58/58 [==============================] - 10s 178ms/step - loss: 0.7177 - accuracy: 0.7054
Epoch 28/30
58/58 [==============================] - 10s 177ms/step - loss: 0.6895 - accuracy: 0.7601
Epoch 29/30
58/58 [==============================] - 10s 176ms/step - loss: 0.7285 - accuracy: 0.7382
Epoch 30/30
58/58 [==============================] - 10s 177ms/step - loss: 0.7687 - accuracy:

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 18s 313ms/step - loss: 4.4273 - accuracy: 0.5246
Epoch 2/30
58/58 [==============================] - 18s 312ms/step - loss: 2.6458 - accuracy: 0.5608
Epoch 3/30
58/58 [==============================] - 18s 313ms/step - loss: 2.0134 - accuracy: 0.6188
Epoch 4/30
58/58 [==============================] - 18s 310ms/step - loss: 1.7101 - accuracy: 0.6900
Epoch 5/30
58/58 [==============================] - 18s 313ms/step - loss: 1.5516 - accuracy: 0.7196
Epoch 6/30
58/58 [==============================] - 18s 314ms/step - loss: 1.3911 - accuracy: 0.7612
Epoch 7/30
58/58 [==============================] - 18s 312ms/step - loss: 1.2943 - accuracy: 0.7514
Epoch 8/30
58/58 [==============================] - 18s 315ms/step - loss: 1.1666 - accuracy: 0.7711
Epoch 9/30
58/58 [==============================] - 18s 311ms/step - loss: 1.0776 - accuracy: 0.8116
Epoch 10/30
58/58 [==============================] - 18s 314ms/step - loss: 1.0344 - accura

58/58 [==============================] - 17s 289ms/step - loss: 0.8971 - accuracy: 0.6561
Epoch 22/30
58/58 [==============================] - 17s 290ms/step - loss: 0.8682 - accuracy: 0.6659
Epoch 23/30
58/58 [==============================] - 17s 289ms/step - loss: 0.8533 - accuracy: 0.7284
Epoch 24/30
58/58 [==============================] - 17s 289ms/step - loss: 1.1258 - accuracy: 0.5367
Epoch 25/30
58/58 [==============================] - 17s 289ms/step - loss: 1.0064 - accuracy: 0.5706
Epoch 26/30
58/58 [==============================] - 17s 288ms/step - loss: 0.9365 - accuracy: 0.6232
Epoch 27/30
58/58 [==============================] - 17s 288ms/step - loss: 0.8898 - accuracy: 0.7065
Epoch 28/30
58/58 [==============================] - 17s 294ms/step - loss: 0.8229 - accuracy: 0.7448
Epoch 29/30
58/58 [==============================] - 17s 291ms/step - loss: 0.7788 - accuracy: 0.7897
Epoch 30/30
58/58 [==============================] - 17s 290ms/step - loss: 0.7405 - accuracy: